<p style="font-size:14pt; font-weight:bold;">Test the theano optimized version of the cost function for the yolo cost

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import theano
from theano import tensor as T
import pickle as pk
import re
from copy import deepcopy
import sys

# image processing
from skimage.io import imread
from skimage.transform import resize

import lasagne
from lasagne.layers import Pool2DLayer, Conv2DLayer, dropout, \
    DenseLayer, InputLayer, get_output, get_all_params
    
import bnr_ml.objectdetect.yolo as yolo
from bnr_ml.utils.helpers import meshgrid2D, softmax

import pdb

In [2]:
reload(yolo)

<module 'bnr_ml.objectdetect.yolo' from '/usr/local/python/bnr_ml/objectdetect/yolo.pyc'>

In [4]:
S = (9,9)
B = 2
C = 4

In [5]:
N = 100
input = 1000*np.random.randn(N,3,200,200).astype(np.float32)
output = np.random.rand(N,B*5 + C, S[0], S[1]).astype(np.float32)
truth = np.random.rand(N,4 + C).astype(np.float32)

In [6]:
input = theano.shared(input)
truth = theano.shared(truth)

In [19]:
net = {}
net['input'] = InputLayer((None,3,200,200))
net['conv1'] = Conv2DLayer(net['input'], 16, (3,3))
net['conv2'] = Conv2DLayer(net['conv1'], 16, (3,3))
net['pool1'] = Pool2DLayer(net['conv2'], (2,2))
net['conv3'] = Conv2DLayer(net['pool1'], 32, (3,3))
net['conv4'] = Conv2DLayer(net['conv3'], 32, (3,3))
net['pool2'] = Pool2DLayer(net['conv4'], (2,2))
net['conv5'] = Conv2DLayer(net['pool2'], 64, (3,3))
net['conv6'] = Conv2DLayer(net['conv5'], 64, (3,3))
net['pool3'] = Pool2DLayer(net['conv6'], (2,2))
net['conv7'] = Conv2DLayer(net['pool3'], 64, (3,3))
net['conv8'] = Conv2DLayer(net['conv7'], 64, (3,3))
net['pool4'] = Pool2DLayer(net['conv8'], (2,2))
net['dense1'] = DenseLayer(dropout(net['pool4'], p=.8), 1000)
net['dense2'] = DenseLayer(dropout(net['dense1'], p=.8), 1000)
net['output'] = DenseLayer(dropout(net['dense2'], p=.5), 5, nonlinearity=lasagne.nonlinearities.softmax)

In [20]:
with open('pretrained_weights.pkl', 'rb') as f:
    weights = pk.load(f)
    lasagne.layers.set_all_param_values(net['output'], weights)

In [21]:
net['dense3'] = DenseLayer(dropout(net['pool4'], p=.8), 2048)
net['output'] = DenseLayer(dropout(net['dense3'], p=.8), (S[0] * S[1]) * (5 * B + C), nonlinearity=None)

In [22]:
yl = yolo.YoloObjectDetector(net, (None, 3, 200, 200), C, S, B) 

In [14]:
def _get_cost_optim(self, output, truth, S, B, C, lmbda_coord=5., lmbda_noobj=0.5, iou_thresh=0.05):
        # calculate height/width of individual cell
        block_height, block_width = 1. / S[0], 1./ S[1]

        # get the offset of each cell
        offset_x, offset_y = meshgrid2D(T.arange(0,1,block_width), T.arange(0,1,block_height))

        # get indices for x,y,w,h,object-ness for easy access
        x_idx, y_idx = T.arange(0,5*B,5), T.arange(1,5*B, 5)
        w_idx, h_idx = T.arange(2,5*B,5), T.arange(3,5*B,5)
        conf_idx = T.arange(4,5*B,5)
        
        # Get position predictions with offsets.
        pred_x = output[:,x_idx] + offset_x.dimshuffle('x','x',0,1)
        pred_y = output[:,y_idx] + offset_y.dimshuffle('x','x',0,1)
        pred_w, pred_h, pred_conf = output[:,w_idx], output[:,h_idx], output[:,conf_idx]
        pred_w, pred_h = T.maximum(pred_w, 0.), T.maximum(pred_h, 0.)

        truth_x, truth_y, truth_w, truth_h = truth[:,0], truth[:,1], truth[:,2], truth[:,3]
        truth_w, truth_h = T.maximum(truth_w, 0.), T.maximum(truth_h, 0.)
        
        # Get intersection region bounding box coordinates
        xi = T.maximum(pred_x, truth_x.dimshuffle(0,'x','x','x'))
        xf = T.minimum(pred_x + pred_w, (truth_x + truth_w).dimshuffle(0,'x','x','x'))
        yi = T.maximum(pred_y, truth_y.dimshuffle(0,'x','x','x'))
        yf = T.minimum(pred_y + pred_h, (truth_y + truth_h).dimshuffle(0,'x','x','x'))
        w, h = T.maximum(xf - xi, 0.), T.maximum(yf - yi, 0.)

        # Calculate iou score for predicted boxes and truth
        isec = w * h
        union = (pred_w * pred_h) + (truth_w * truth_h).dimshuffle(0,'x','x','x') - isec
        iou = T.maximum(isec/union, 0.)

        # Get index matrix representing max along the 1st dimension for the iou score (reps 'responsible' box).
        maxval_idx, _ = meshgrid2D(T.arange(B), T.arange(truth.shape[0]))
        maxval_idx = maxval_idx.dimshuffle(0,1,'x','x')
        maxval_idx = T.repeat(T.repeat(maxval_idx,S[0],2),S[1],3)
        
        is_max = T.eq(maxval_idx, iou.argmax(axis=1).dimshuffle(0,'x',1,2))
        is_not_max = T.neq(maxval_idx, iou.argmax(axis=1).dimshuffle(0,'x',1,2))
        
        # Get matrix for the width/height of each cell
        width, height = T.ones(S) / S[1], T.ones(S) / S[0]
        width, height = width.dimshuffle('x',0,1), height.dimshuffle('x',0,1)
        offset_x, offset_y = offset_x.dimshuffle('x',0,1), offset_y.dimshuffle('x',0,1)
        
        # Get bounding box for intersection between CELL and ground truth box.
        xi = T.maximum(offset_x, truth_x.dimshuffle(0,'x','x'))
        xf = T.minimum(offset_x + width, (truth_x + truth_w).dimshuffle(0,'x','x'))
        yi = T.maximum(offset_y, truth_y.dimshuffle(0,'x','x'))
        yf = T.minimum(offset_y + height, (truth_y + truth_h).dimshuffle(0,'x','x'))
        w, h = T.maximum(xf - xi, 0.), T.maximum(yf - yi, 0.)

        # Calculate iou score for the cell.
        isec = (xf - xi) * (yf - yi)
        union = (width * height) + (truth_w* truth_h).dimshuffle(0,'x','x') - isec
        iou_cell = T.maximum(isec/union, 0.)
        
        # Get logical matrix representing minimum iou score for cell to be considered overlapping ground truth.
        is_inter = (iou_cell > iou_thresh).dimshuffle(0,'x',1,2)
        
        # repeat "cell overlaps" logical matrix for the number of classes.
        is_inter = T.repeat(is_inter, C, axis=1)
        
        # repeat the ground truth for class probabilities for each cell.
        clspred_truth = T.repeat(T.repeat(truth[:,-C:].dimshuffle(0,1,'x','x'), S[0], axis=2), S[1], axis=3)
        
        # calculate cost
        cost = T.sum((pred_conf - iou)[is_max.nonzero()]**2) + \
            lmbda_noobj * T.sum((pred_conf[is_not_max.nonzero()])**2) + \
            lmbda_coord * T.sum((pred_x[is_max.nonzero()].reshape((truth.shape[0],-1)) - truth_x.dimshuffle(0,'x'))**2) + \
            lmbda_coord * T.sum((pred_y[is_max.nonzero()].reshape((truth.shape[0],-1)) - truth_y.dimshuffle(0,'x'))**2) + \
            lmbda_coord * T.sum((pred_w[is_max.nonzero()].reshape((truth.shape[0],-1)).sqrt() - truth_w.dimshuffle(0,'x').sqrt())**2) + \
            lmbda_coord * T.sum((pred_h[is_max.nonzero()].reshape((truth.shape[0],-1)).sqrt() - truth_h.dimshuffle(0,'x').sqrt())**2) + \
            T.sum((output[:,-C:][is_inter.nonzero()] - clspred_truth[is_inter.nonzero()])**2)
#         pdb.set_trace()
        return cost

In [15]:
_get_cost_optim(yl, yl.output_test, truth, S, B, C)

> <ipython-input-14-bd92ee035b9e>(71)_get_cost_optim()
-> return cost
(Pdb) cost.eval()
array(171283936.0, dtype=float32)
(Pdb) q


BdbQuit: 

In [ ]:
def manual_cost(output, truth, C, S, B, lmbda_coord=5., lmbda_noobj=.5, thresh = .05):
    output, truth = np.copy(output), np.copy(truth)
    def calc_iou(b1, b2):
        xi = np.maximum(b1[0], b2[0])
        xf = np.minimum(b1[0]+b1[2], b2[0]+b2[2])
        yi = np.maximum(b1[1], b2[1])
        yf = np.minimum(b1[1]+b1[3], b2[1]+b2[3])
        w, h = np.maximum(0., xf - xi), np.maximum(0., yf - yi)
        isec = np.maximum(w * h, 0.)
        union = (b1[2]*b1[3]) + (b2[2]*b2[3]) - isec
        iou = isec / union
        return iou
    
    cost = 0.
    xshift, yshift = 1./S[1], 1./S[0]
    for i in range(S[0]):
        for j in range(S[1]):
            iou_score_per_box = np.zeros((B,))
            for k in range(B):
                reg_truth = truth[:4]
                reg_box = output[k*5:k*5 + 4, i, j]
                reg_box[0] += j * xshift
                reg_box[1] += i * yshift
                iou_score_per_box[k] = calc_iou(reg_truth, reg_box)
            
            idx_resp = np.argmax(iou_score_per_box)
            
            for k in range(B):
                reg_fact = lmbda_noobj
                if k == idx_resp:
                    reg_fact = 1.
                cost += reg_fact * (output[k*B + 4, i, j] - iou_score_per_box[k])**2
        
            reg_cell = np.asarray([j * xshift, i * yshift, xshift, yshift])
            iou_cell = calc_iou(reg_cell, truth[:4])
            
            if iou_cell > thresh:
                cost += ((output[-C:, i, j] - truth[-C:])**2).sum()
                
            cost += lmbda_coord * ((output[idx_resp*5:idx_resp*5+2,i,j] - truth[:2])**2).sum()
            cost += lmbda_coord * ((np.sqrt(output[idx_resp*5+2:idx_resp*5+4,i,j]) - np.sqrt(truth[2:4]))**2).sum()
    
    return cost
            

In [43]:
N = 100
input = 10000*np.random.randn(N,3,200,200).astype(np.float32)
output = np.random.rand(N,B*5 + C, S[0], S[1]).astype(np.float32)
truth = np.random.rand(N,4 + C).astype(np.float32)

In [23]:
target = T.matrix('target')
art_output = T.tensor4('art_output')

In [24]:
out_fn = theano.function([yl.input], yl.output_test, allow_input_downcast=True)

In [26]:
fn = theano.function([yl.input, target], yl._get_cost_optim(yl.output_test, target, S, B, C), allow_input_downcast=True)

In [42]:
fn(input, truth)

array(14131851264.0, dtype=float32)

In [ ]:
# fn2 = theano.function([art_output, target], get_cost_optim(yl, art_output, target, S, B, C), allow_input_downcast=True)

In [ ]:
art_output = iput

In [ ]:
fn(iput, truth)

In [ ]:
manual_cost(out_fn(iput)[0], truth[0], C, S, B)

In [ ]:
# Create artificially correct answer
truth = np.random.rand(1, 4 + C)
true_column = np.concatenate((truth[[0],:4], [[1.]], [[0.,0.,.0,.0]], [[0.]], truth[[0],-C:]), axis=1).reshape((1,5*B + C, 1, 1))
true_column = np.repeat(true_column, S[0], axis=2)
true_column = np.repeat(true_column, S[1], axis=3)

offset_x, offset_y = np.meshgrid(np.arange(0.,1.,1./9), np.arange(0.,1.,1./9))
true_column[:,[0,5]] -= offset_x.reshape((-1,1,9,9))
true_column[:,[1,6]] -= offset_y.reshape((-1,1,9,9))

In [ ]:
fn2(true_column, truth)

In [ ]:
manual_cost(true_column[0], truth[0], C, S, B)